In [ ]:
import requests
# import import_ipynb
from ipywidgets import widgets, interact_manual
from googleapi import get_sentiment, key_phrase
from dotenv import load_dotenv
import os


def config():
    load_dotenv()


def get_lyrics(track, artist):
    # Get thr lyrics of a song
    url = 'https://api.musixmatch.com/ws/1.1/matcher.lyrics.get'

    params = {
        "q_track": track,
        "q_artist": artist,
        "apikey": os.getenv("musix_api_key")
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    response = response.json()
    status = response['message']['header']['status_code']
    if 200 <= status <= 299:
        lyrics = response['message']['body']['lyrics']['lyrics_body']
        return lyrics
    else:
        return 'error'


@interact_manual(Song='', Artist='')
def main(Song, Artist):
    load_dotenv()
    #     'First world problem', 'Weird Al Yankovic'
    # get the lyrics of the song using Musixmatch's api
    data = get_lyrics(Song, Artist)
    if data == 'error':
        print("Invalid Song or Artist Name!")
    else:
        lyrics = data
        # extract key phrases using microsfot azure text api
        phrases = key_phrase(lyrics)
        # Get the sentiment of the song
        sentiment = get_sentiment(lyrics)

        # put all key phrases into a list
        all_phrases = ''
        for phrase in phrases['documents'][0]['keyPhrases']:

            # if the key phrase is the last element in the list add it with out a comma
            if phrase == phrases['documents'][0]['keyPhrases'][-1]:
                all_phrases += (f"and {phrase.title()}. ")
            # add all the key phrases into the list
            else:
                all_phrases += (f"{phrase.title()}, ")

        # print the gist of the song
        print(f"{Song} by {Artist} is a {sentiment} song about {all_phrases}")


